In [2]:
import requests
from bs4 import BeautifulSoup

In [44]:
# Retrieve content from the website
imdb_url = "https://www.imdb.com"
search_urls = ["https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&ref_=adv_prv",
               "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=51&ref_=adv_nxt", 
               "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=101&ref_=adv_nxt", 
               "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=151&ref_=adv_nxt", 
               "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=201&ref_=adv_nxt"]
headers = {"Accept-Language": "en-US,en;q=0.5"}

movies = []
for url in search_urls:
    response = requests.get(url, headers=headers)
    content = response.content
    soup = BeautifulSoup(content,"html.parser")

    # Scrape the data using HTML tags
    movies_list = soup.find_all("div", attrs={"class": "lister-item-content"})
    for movie in movies_list:
        df = {}
        df["Movie Title"] = movie.find("a").text
        df["Movie URL"] = imdb_url + movie.find("a")['href']
        df["Release Year"] = movie.find("span", attrs={"class": "lister-item-year"}).text[-5:-1]
        df["Rank"] = movie.find_all("span", attrs={"name": "nv"})[-1].text[1:]
        df["Ratings"] = movie.find("strong").text
        df["Votes"] = movie.find_all("span", attrs={"name": "nv"})[0].text
        movies.append(df)

In [60]:
import pandas as pd
dataframe = pd.DataFrame(movies)
dataframe["Release Year"] = pd.to_datetime(dataframe["Release Year"])
dataframe['Rank'] = dataframe['Rank'].astype('int')
dataframe['Ratings'] = dataframe['Ratings'].astype('float')
dataframe['Votes'] = dataframe['Votes'].str.replace(',', '').astype('int')
dataframe.sort_values(by='Rank', inplace=True)
dataframe.reset_index(drop=True, inplace=True)
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Movie Title   250 non-null    object        
 1   Movie URL     250 non-null    object        
 2   Release Year  250 non-null    datetime64[ns]
 3   Rank          250 non-null    int32         
 4   Ratings       250 non-null    float64       
 5   Votes         250 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2), object(2)
memory usage: 9.9+ KB


In [59]:
# print the dataframes
print(dataframe)

# save the result in CSV format
dataframe.to_csv('imdb_top_250_movies.csv')

                                        Movie Title  \
0                          The Shawshank Redemption   
1                                     The Godfather   
2                            The Godfather: Part II   
3                                   The Dark Knight   
4                                      12 Angry Men   
..                                              ...   
245  Neon Genesis Evangelion: The End of Evangelion   
246                                   Andrei Rublev   
247                               Nights of Cabiria   
248                                 Soorarai Pottru   
249                           The Battle of Algiers   

                                  Movie URL Release Year  Rank  Ratings  \
0     https://www.imdb.com/title/tt0111161/   1994-01-01     1      9.3   
1     https://www.imdb.com/title/tt0068646/   1972-01-01     2      9.2   
2     https://www.imdb.com/title/tt0071562/   1974-01-01     3      9.0   
3     https://www.imdb.com/title/tt0468